In [3]:
from glob import glob
import os

files = glob('*/*.mp3')
files = [f for f in files if 'fail' not in f.lower() and 'dedup' not in f and 'v2' not in f]
len(files)

515728

In [4]:
folders = set()
for f in files:
    folders.add(os.path.split(f)[0])
    
folders

{'mallm',
 'partition-instructions-part-0',
 'partition-instructions-part-1',
 'partition-instructions-part-10',
 'partition-instructions-part-11',
 'partition-instructions-part-12',
 'partition-instructions-part-13',
 'partition-instructions-part-14',
 'partition-instructions-part-15',
 'partition-instructions-part-16',
 'partition-instructions-part-17',
 'partition-instructions-part-2',
 'partition-instructions-part-3',
 'partition-instructions-part-4',
 'partition-instructions-part-5',
 'partition-instructions-part-6',
 'partition-instructions-part-7',
 'partition-instructions-part-8',
 'partition-instructions-part-9',
 'short-coding-0',
 'short-coding-1',
 'short-coding-2',
 'tatabahasa'}

In [5]:
import zipfile
import mp
import time
from huggingface_hub import HfFileSystem
from huggingface_hub import HfApi
from tqdm import tqdm
api = HfApi()

partition_size = 5e+9

In [6]:
def loop(files):
    files, index = files
    current_index = 0
    api = HfApi()
    fs = HfFileSystem()
    total = 0
    temp = []
    for i in tqdm(range(len(files))):
        s = os.stat(files[i]).st_size
        if s + total >= partition_size:
            part_name = f"speech-instructions-{index}-{current_index}.zip"
                
            with zipfile.ZipFile(part_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
                for f in temp:
                    zipf.write(f, arcname=f)

            while True:
                try:
                    api.upload_file(
                        path_or_fileobj=part_name,
                        path_in_repo=part_name,
                        repo_id="malaysia-ai/STT-Whisper",
                        repo_type="dataset",
                    )
                    break
                except:
                    time.sleep(60)

            os.remove(part_name)
            
            current_index += 1
            temp = [files[i]]
            total = s
        else:
            temp.append(files[i])
            total += s
        
    if len(temp):
        part_name = f"speech-instructions-{index}-{current_index}.zip"

        with zipfile.ZipFile(part_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for f in temp:
                zipf.write(f, arcname=f)

        while True:
            try:
                api.upload_file(
                    path_or_fileobj=part_name,
                    path_in_repo=part_name,
                    repo_id="malaysia-ai/STT-Whisper",
                    repo_type="dataset",
                )
                break
            except:
                time.sleep(60)

        os.remove(part_name)

In [7]:
loop((files[:1000], 0))

100%|████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 18585.10it/s]


speech-instructions-0-0.zip:   0%|          | 0.00/42.7M [00:00<?, ?B/s]

In [9]:
mp.multiprocessing(files, loop, cores = 10, returned = False)